
# ENEM 2023 — 03 · Rótulos do Dicionário & Export Rotulada

Objetivo: **ler o dicionário .xlsx**, aplicar rótulos legíveis às variáveis categóricas (ex.: `TP_ESCOLA`, `Q001–Q006`), 
e **salvar uma versão rotulada** do dataset para uso nos gráficos e no relatório.

- Sem seaborn; apenas **matplotlib**.
- Salva arquivos em:
  - `../data/interim/enem_2023_rotulado.parquet`
  - `../reports/figures/*`
  - `../reports/tabelas/dicionario_mapeamentos.csv`


In [ ]:

# Imports
import os
from pathlib import Path
import re
import pandas as pd
import matplotlib.pyplot as plt

# Caminhos (ajuste se necessário)
PARQUET_IN = Path("../data/interim/enem_2023.parquet").resolve()
DICT_PATH  = Path("../data/interim/unzipped_2023/DICIONÁRIO/Dicionário_Microdados_Enem_2023.xlsx").resolve()

PARQUET_OUT = Path("../data/interim/enem_2023_rotulado.parquet").resolve()
FIG_DIR     = Path("../reports/figures"); FIG_DIR.mkdir(parents=True, exist_ok=True)
TAB_DIR     = Path("../reports/tabelas"); TAB_DIR.mkdir(parents=True, exist_ok=True)

print("Usando:", PARQUET_IN)
print("Dicionário:", DICT_PATH)


## 1) Helper para ler o dicionário (.xlsx) e extrair rótulos

In [ ]:

def read_enem_dictionary(dict_path: str | Path, sheet_name: str = "MICRODADOS_ENEM_2023"):
    df = pd.read_excel(dict_path, sheet_name=sheet_name, header=2)
    # localizar colunas por palavras-chave (tolerante a acentos/maiúsculas)
    def find_col(keyword):
        for c in df.columns:
            if keyword.lower() in str(c).lower():
                return c
        return None
    col_var = find_col("variável")
    col_desc = find_col("descrição")
    col_cat = find_col("valor") or find_col("categ")
    if not col_var or not col_desc:
        raise ValueError("Não foi possível identificar colunas de variável/descrição no dicionário.")
    descriptions, categories = {}, {}
    for _, row in df.iterrows():
        var = str(row[col_var]).strip().upper()
        if not var or var == "NAN":
            continue
        descriptions[var] = str(row[col_desc]).strip()
        cats_raw = str(row[col_cat]) if col_cat else ""
        pairs = []
        for part in re.split(r"[;\n]+", cats_raw):
            m = re.match(r"\s*([A-Za-z0-9]+)\s*[-=:\u2013]\s*(.+)", part)
            if m:
                code, label = m.group(1).strip(), m.group(2).strip()
                pairs.append((code, label))
        if pairs:
            categories[var] = {k: v for k, v in pairs}
    return descriptions, categories


## 2) Carregar base e dicionário

In [ ]:

df = pd.read_parquet(PARQUET_IN)
descriptions, categories = read_enem_dictionary(DICT_PATH)
print("Linhas:", len(df), "| Variáveis descritas:", len(descriptions), "| Com categorias:", len(categories))
list(categories.keys())[:10]


## 3) Aplicar rótulos às variáveis importantes

In [ ]:

def apply_labels(df, var, cats: dict, order_codes: list[str] | None = None, new_col=None):
    if not cats:
        return df
    target = new_col or var
    s = df[var].astype("string")
    # ordem: a que vier do dicionário (ou a fornecida)
    order = order_codes or list(cats.keys())
    labeled = s.map(cats).astype("category").cat.set_categories([cats[k] for k in order], ordered=True)
    df[target] = labeled
    return df

# TP_ESCOLA (fallback se não vier do dicionário)
tp_fallback = {"1": "Não respondeu", "2": "Pública", "3": "Privada", "4": "Exterior"}
tp_cats = categories.get("TP_ESCOLA", tp_fallback)
tp_order = ["2","3","1","4"]  # Pública, Privada, Não resp., Exterior
if "TP_ESCOLA" in df.columns:
    df["TP_ESCOLA"] = df["TP_ESCOLA"].astype("Int64").astype("string")  # normaliza
    df = apply_labels(df, "TP_ESCOLA", tp_cats, order_codes=tp_order, new_col="TP_ESCOLA_rot")

# Q001–Q006, Q022, Q024, Q025 (se existirem)
for q in ["Q001","Q002","Q003","Q004","Q005","Q006","Q022","Q024","Q025"]:
    if q in df.columns:
        cats = categories.get(q, {})
        if cats:
            # ordem A..Z como padrão
            order = sorted(cats.keys(), key=lambda x: x)
            df[q] = df[q].astype("string")
            df = apply_labels(df, q, cats, order_codes=order, new_col=f"{q}_rot")
        else:
            # sem rótulo: mantém a original
            pass

df.head(3)


## 4) Salvar versão rotulada (Parquet)

In [ ]:

df.to_parquet(PARQUET_OUT, index=False)
print("Parquet rotulado salvo em:", PARQUET_OUT)


## 5) Figuras com rótulos — Boxplot por tipo de escola

In [ ]:

if "TP_ESCOLA_rot" in df.columns:
    plt.figure(figsize=(7,4))
    df.boxplot(column="NOTA_MEDIA_5", by="TP_ESCOLA_rot", grid=False)
    plt.title("NOTA_MEDIA_5 por Tipo de Escola — ENEM 2023 (rotulado)")
    plt.suptitle("")
    plt.xlabel("")
    plt.ylabel("Média das 5 provas")
    plt.tight_layout()
    out = FIG_DIR / "boxplot_media5_por_tp_escola_rotulado.png"
    plt.savefig(out, dpi=200, bbox_inches="tight")
    plt.show()
    print("Figura salva:", out)
else:
    print("TP_ESCOLA_rot não disponível.")


## 6) Tabela de mapeamentos (para relatório)

In [ ]:

# Exporta um CSV longo com var, código e rótulo
rows = []
for var, mapping in categories.items():
    for code, label in mapping.items():
        rows.append({"variavel": var, "codigo": code, "rotulo": label})
map_df = pd.DataFrame(rows).sort_values(["variavel","codigo"])
out_csv = TAB_DIR / "dicionario_mapeamentos.csv"
map_df.to_csv(out_csv, index=False)
print("Mapeamentos salvos em:", out_csv)
map_df.head(12)


## 7) Tendência por renda (Q006_rot) com rótulos do dicionário

In [ ]:

if "Q006_rot" in df.columns and "NOTA_MEDIA_5" in df.columns:
    tmp = df.dropna(subset=["Q006_rot","NOTA_MEDIA_5"]).copy()
    # Usa a ordem categórica já aplicada
    cats = list(tmp["Q006_rot"].cat.categories)
    m = tmp.groupby("Q006_rot")["NOTA_MEDIA_5"].mean().reindex(cats)
    plt.figure(figsize=(8,4))
    plt.plot(range(len(m)), m.values, marker="o")
    plt.title("Média das 5 provas por faixa de renda (Q006) — Rotulada")
    plt.xlabel("Faixa de renda (Q006)")
    plt.ylabel("Média NOTA_MEDIA_5")
    plt.xticks(range(len(m)), cats, rotation=0, ha="center")
    # quebra rótulos longos
    import textwrap
    ax = plt.gca()
    ax.set_xticklabels(["
".join(textwrap.wrap(t, 20)) for t in cats])
    plt.tight_layout()
    out = FIG_DIR / "linha_media5_por_q006_rotulada.png"
    plt.savefig(out, dpi=200, bbox_inches="tight")
    plt.show()
    print("Figura salva:", out)
else:
    print("Q006_rot ou NOTA_MEDIA_5 indisponíveis.")
